In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
# from google.colab.patches import cv2_imshow
import urllib.request
from time import perf_counter

In [2]:
# req = urllib.request.urlopen('https://placekitten.com/512/512')
req = urllib.request.urlopen('https://www.bioparcvalencia.es/wp-content/uploads/2017/06/ficha-animal-bioparc-valencia-chimpance.jpg')
arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
img = cv2.imdecode(arr,-1)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [3]:
def completeConvolution(img, kernel):
    kernel = np.flip(kernel)
    nrows, ncols = img.shape
    krows, kcols = kernel.shape
    pad_size = (krows - 1) // 2
    padded_img = np.pad(img, pad_size, mode='constant')
    output = np.zeros((nrows, ncols))
    for i in range(nrows):
        for j in range(ncols):
            output[i, j] = np.sum(
                padded_img[i:i+krows, j:j+kcols] * kernel)
    return output


def convolucion4F(img, kernel):
  """
  Regresa imagen con operación de ventana, 
  args: imagengray y matriz kernel 
  """
  kernel = np.flip(kernel)
  nr = img.shape[0]
  nc = img.shape[1]

  nk = kernel.shape[0]//2
  nl = kernel.shape[1]//2
  
  out = np.zeros(img.shape)

  # con 4 ciclos for
  for r in range(nr):
    for c in range(nc):
      for k in range(-nk, nk+1):
        for l in range(-nl, nl+1):
          if k+r >= 0 and l+c >=0 and k+r<nr and l+c<nc:
            out[r,c] += kernel[k+nk,l+nk]*img[r+k,c+l]
            # print(out)
  return out

In [4]:
x=np.array([[1,2,3,4,5,6,7,8,9,10],
            [1,2,3,4,5,6,7,8,9,10],
            [1,2,3,4,5,6,7,8,9,10],
            [1,2,3,4,5,6,7,8,9,10],
            [1,2,3,4,5,6,7,8,9,10],
            [1,2,3,4,5,6,7,8,9,10],
            [1,2,3,4,5,6,7,8,9,10],
            [1,2,3,4,5,6,7,8,9,10],
            [1,2,3,4,5,6,7,8,9,10],
            [1,2,3,4,5,6,7,8,9,10]])

h=np.array([
  [1,2,1],
  [2,4,2],
  [1,2,1]
])

# x = np.array([[1, 2, 0, 0],
#               [5, 3, 0, 4],
#               [0, 0, 0, 7],
#               [9, 3, 0, 0]])

# h = np.array([[1,1,1],[1,1,0],[1,0,0]])

# x = np.array([[1, 2, 3],
#                 [4, 5, 6],
#                 [7, 8, 9]])
# h = np.array([[1, 0, -1],
#                    [2, 0, -2],
#                    [1, 0, -1]])

convImg = completeConvolution(x, h)
vimg= convolucion4F(x, h)


print('2 ciclos')
print(convImg)

print('4 ciclos')
print(vimg)

print(convImg==vimg)


2 ciclos
[[ 12.  24.  36.  48.  60.  72.  84.  96. 108.  87.]
 [ 16.  32.  48.  64.  80.  96. 112. 128. 144. 116.]
 [ 16.  32.  48.  64.  80.  96. 112. 128. 144. 116.]
 [ 16.  32.  48.  64.  80.  96. 112. 128. 144. 116.]
 [ 16.  32.  48.  64.  80.  96. 112. 128. 144. 116.]
 [ 16.  32.  48.  64.  80.  96. 112. 128. 144. 116.]
 [ 16.  32.  48.  64.  80.  96. 112. 128. 144. 116.]
 [ 16.  32.  48.  64.  80.  96. 112. 128. 144. 116.]
 [ 16.  32.  48.  64.  80.  96. 112. 128. 144. 116.]
 [ 12.  24.  36.  48.  60.  72.  84.  96. 108.  87.]]
4 ciclos
[[ 12.  24.  36.  48.  60.  72.  84.  96. 108.  87.]
 [ 16.  32.  48.  64.  80.  96. 112. 128. 144. 116.]
 [ 16.  32.  48.  64.  80.  96. 112. 128. 144. 116.]
 [ 16.  32.  48.  64.  80.  96. 112. 128. 144. 116.]
 [ 16.  32.  48.  64.  80.  96. 112. 128. 144. 116.]
 [ 16.  32.  48.  64.  80.  96. 112. 128. 144. 116.]
 [ 16.  32.  48.  64.  80.  96. 112. 128. 144. 116.]
 [ 16.  32.  48.  64.  80.  96. 112. 128. 144. 116.]
 [ 16.  32.  48.  64.  80. 

In [5]:

def convDesacoplada(img, kernel):
  kernel = kernel[::-1]
  nr = img.shape[0]
  nc = img.shape[1]
  output = np.zeros(img.shape)
  m = len(kernel) // 2

  for l in range(nr):
    for n in range(nc):
      for k in range(-m, m+1):
          if n + k < 0 or n + k >= nc:
              continue
          # print(l,n,k, img[l,n+k], kernel[k+m])    
          output[l,n] += img[l,n+k] * kernel[k+m]
  # print( output)

  output2 = np.zeros(img.shape)
  for l in range(nr):
    for n in range(nc):
      for k in range(-m, m+1):
          if l + k < 0 or l + k >= nr:
              continue
          # print(l,n,k, output[l+k,n], kernel[k+m])    
          output2[l,n] += output[l+k,n]* kernel[k+m]
          # print( output)

  return output2

In [6]:
h= np.array([1,2,1])
x=np.array([[1,2,3,4,5,6,7,8,9,10],
            [1,2,3,4,5,6,7,8,9,10],
            [1,2,3,4,5,6,7,8,9,10]])
print(convDesacoplada(x,h))

[[ 12.  24.  36.  48.  60.  72.  84.  96. 108.  87.]
 [ 16.  32.  48.  64.  80.  96. 112. 128. 144. 116.]
 [ 12.  24.  36.  48.  60.  72.  84.  96. 108.  87.]]


In [7]:
x=np.array([[1,2,3,4,5,6,7,8,9,10],
            [1,2,3,4,5,6,7,8,9,10],
            [1,2,3,4,5,6,7,8,9,10],
            [1,2,3,4,5,6,7,8,9,10],
            [1,2,3,4,5,6,7,8,9,10],
            [1,2,3,4,5,6,7,8,9,10],
            [1,2,3,4,5,6,7,8,9,10],
            [1,2,3,4,5,6,7,8,9,10],
            [1,2,3,4,5,6,7,8,9,10],
            [1,2,3,4,5,6,7,8,9,10]])

h =np.array([1,2,1])

def convolveDesacopladaDot(img, kernel):
    kernel = np.flip(kernel)
    nrows, ncols = img.shape
    k_size = kernel.size
    # krows, kcols = kernel.shape
    pad_size = (k_size - 1) // 2
    padded_img = np.pad(img, pad_size, mode='constant')
    # print(padded_img)
    output = np.zeros((nrows, ncols))
    for i in range(ncols):
      output[i,: ]= convolveDot(img[i, :], kernel)
    # print(output)
    output2 = np.zeros((nrows, ncols))
    for j in range(nrows):
      output2[:, j ]= convolveDot(output[ :, j], kernel)
      # print(img[:, j])
    print(output2)

def convolveDot(signal, kernel):
    ker_size = kernel.size
    sig_size = signal.size
    kernel = kernel[::-1]
    pad_size = (ker_size - 1) // 2
    padded_signal = np.pad(signal, pad_size, mode='constant')
    output = np.zeros(sig_size)
    output = [np.dot(padded_signal[k:k+ker_size], kernel) for k in range(sig_size)]
    # print(f"a: {output}" )
    # for k in range(sig_size):
    #     output[k] = np.dot(padded_signal[k:k+ker_size], kernel)
    # print(output)
    return output

convolveDesacopladaDot(x,h)
# print(x)

[[ 12.  24.  36.  48.  60.  72.  84.  96. 108.  87.]
 [ 16.  32.  48.  64.  80.  96. 112. 128. 144. 116.]
 [ 16.  32.  48.  64.  80.  96. 112. 128. 144. 116.]
 [ 16.  32.  48.  64.  80.  96. 112. 128. 144. 116.]
 [ 16.  32.  48.  64.  80.  96. 112. 128. 144. 116.]
 [ 16.  32.  48.  64.  80.  96. 112. 128. 144. 116.]
 [ 16.  32.  48.  64.  80.  96. 112. 128. 144. 116.]
 [ 16.  32.  48.  64.  80.  96. 112. 128. 144. 116.]
 [ 16.  32.  48.  64.  80.  96. 112. 128. 144. 116.]
 [ 12.  24.  36.  48.  60.  72.  84.  96. 108.  87.]]


In [8]:
from scipy.signal import convolve2d

scipi = convolve2d(x,h, mode='same')

ValueError: convolve2d inputs must both be 2-D arrays